# Benasque Spring School on Near-Term Quantum Computing
## Quantum Circuit simulation with Tensor Networks

Let's start with creating a simple quantum circuit:

In [ ]:
using Quac

n_qubits = 8
circ = Circuit(n_qubits)

In [ ]:
push!(circ, H(1))
circ

Now let's create a simple quantum circuits with `10` qubits and `4` layers of random gates.

In [ ]:
using Random

n = 10
depth = 8

# Define the quantum circuit
circ = Circuit(n)
gates = [Swap, CX, CY, CZ, CRx, CRy, CRz]

Random.seed!(1)

# We want to apply some entangling gates
for _ in 1:depth
    for i in 1:n
        gate = rand(gates)
        j = rand(setdiff(1:n, i))
        push!(circ, gate(i, j))
    end
end

circ

### Exact simulation

Now that we have created a circuit, let's convert it to a tensor network with `Tenet`

In [ ]:
using Qrochet

U = Quantum(circ)

In [ ]:
using Tenet

tn = TensorNetwork(U)

We can use `CairoMakie` to visualize the tensor network:

In [ ]:
using CairoMakie
using GraphMakie
using NetworkLayout

plot(tn, layout=Stress())

We get a `TensorNetwork` object that contains the tensor network representation of the circuit. It has $2N$ open indices, where $N$ is the number of qubits in the circuit, since we need to connect the input and output  qubits of the circuit to the tensor network. We will now contract the tensor network with the initial and final states to get the expectation value of the circuit.

In [ ]:
ψᵢₙ = Product(fill([1, 0], n))
ψₒᵤₜ = Product(fill([1, 0], n))

qtn = merge(Quantum(ψᵢₙ), U, Quantum(ψₒᵤₜ)')
tn = TensorNetwork(qtn)

plot(tn, layout=Stress())

We can use `EinExprs.jl` to see the contraction path of the tensor network:

In [ ]:
using EinExprs

path = einexpr(tn, optimizer=Greedy())
@show total_flops = mapreduce(flops, +, Branches(path))
@show max_dims = maximum(ndims, Branches(path));

We can also plot the contraction path:

In [ ]:
plot(path, layout=Stress())

Okay! Let's contract the circuit:

In [ ]:
contract(TensorNetwork(qtn); path)

### Approximate simulation

Let's create a more complex circuit that is made of two-qubit local gates:

In [ ]:
Random.seed!(1)

n = 50
depth = 14

# Define the quantum circuit
gates = [Rxx, Ryy, Rzz]
circ = Circuit(n)
for i in 1:depth
    for j in 1:n-1
        rand_gate = rand(gates)
        push!(circ, rand_gate(j, j+1; θ=rand()*2π))
    end
end

We define the `ψᵢₙ ` and `ψₒᵤₜ` product states:

In [ ]:
ψᵢₙ = Product(fill([1, 0], n))
ψₒᵤₜ = Product(fill([1, 0], n))

plot(TensorNetwork(ψᵢₙ))

In [ ]:
U = Quantum(circ)

qtn = merge(Quantum(ψᵢₙ), U, Quantum(ψₒᵤₜ)')
tn = TensorNetwork(qtn)

plot(tn; layout=Spring())

In [ ]:
path = einexpr(tn, optimizer=Greedy())

@show total_flops = mapreduce(flops, +, Branches(path))
@show max_dims = maximum(ndims, Branches(path));

In [ ]:
plot(path, layout=Stress())

Much larger circuit!! We need an approximation! We will use MPS

In [ ]:
@time c = contract(tn; path) # Exact result

In [ ]:
ψᵢₙ = convert(Chain, Product(fill([1, 0], n)))
ψₒᵤₜ = Product(fill([1, 0], n))

@time begin
    # Apply gates using `evolve!` function  
    for gate in circ
        evolve!(ψᵢₙ, gate; maxdim=2, renormalize=false)
    end

    overlap(ψᵢₙ, ψₒᵤₜ)
end

In [ ]:
using LinearAlgebra

@show fidelity = norm(ψᵢₙ);

We can limit the bond dimension of the MPS to the computational complexity of the operation. A limit in the bond dimension can be thought as a limit in the entanglement of the state. The bond dimension is a trade-off between accuracy and computational cost. The larger the bond dimension, the more accurate the approximation, but the more expensive the computation.

Let's use a reduced bond dimension $\chi$ and compare it with the exact result.

In [ ]:
# Let's evolve for different bond dimensions χ and compare to the exact result
results = ComplexF32[]
χ_list = [2, 4, 8, 16, 32, 64, 128]
ψₒᵤₜ = Product(fill([1, 0], n))

for χ in χ_list
    @show χ
    ψᵢₙ = convert(Chain, Product(fill([1, 0], n)))
    canonize!(ψᵢₙ)

    for gate in circ
        evolve!(ψᵢₙ, gate; maxdim=χ, iscanonical=true, renormalize=true)
    end

    push!(results, only(overlap(ψᵢₙ, ψₒᵤₜ)))
end

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel="χ", ylabel="|Amplitude|", title="Expectation value", xscale=log2)

scatterlines!(ax, Float32.(χ_list), abs.(results), color=:red, linestyle=:dash, marker=:circle, label="Approximation")
lines!(ax, Float32.(χ_list), abs(only(c))*ones(length(χ_list)), color=:blue, linestyle=:solid, label="Exact value")

# Adjust the legend manually
legend = Legend(fig[1, 2], ax, valign=:top)
fig[1, 2] = legend

fig # Display the figure

### Time Evolution of a Matrix Product State

We will use the `Time Evolving Block Decimation (TEBD)` algorithm to simulate the time evolution of a Matrix Product State (MPS) circuit.

##### Transverse-field Ising Model

The Hamiltonian of transverse field Ising model is given by:
$H = -\sum_{\braket{i,j}} \sigma_{X}^{(i)} \sigma_{X}^{(j)} + \lambda \sum_i \sigma_Z^{(i)}$

We will simulate the time evolution of a quantum state under this Hamiltonian, and we will measure some `observables` of the system.

In the `TEBD` algorithm we will use the Suzuki-Trotter decomposition to the first order, which approximates $e^{-iHt}$ as $e^{-iH_1 \Delta t} e^{-iH_2 \Delta t} ... e^{-iH_n \Delta t}$, where $H = H_1 + H_2 + ... + H_n$, where $\Delta t$ is the time step.

In [ ]:
observables = Dense.([Z(5)])

n = 10
δₜ = 0.1
timesteps = 100

We define two gates that will be used. The `trotter_XX` gate is a Trotterized version of the XX gate, and the `trotter_Z` gate is a Trotterized version of the Z gate.

In [ ]:
function trotter_XX(i,j; δₜ=δₜ)
    mat = kron(Matrix(X()), Matrix(X()))
    mat = cis(δₜ * mat)
    mat = reshape(mat,2,2,2,2)
    Dense(Qrochet.Operator(), mat; sites=Site[Site(i), Site(j), Site(i, dual=true), Site(j, dual=true)])
end

function trotter_Z(i; λ, δₜ=δt)
    mat = Matrix(Z())
    mat = cis(- λ * δₜ * mat)

    Dense(Qrochet.Operator(), mat; sites=Site[Site(i), Site(i, dual=true)])
end

We start the quench from a product state where all qubits are in the $|+\rangle$ state.

In [ ]:
ket₊ = 1/√2 * [1, 1]
ψ = Product(fill(ket₊, n))
ψ = convert(Chain, ψ)

plot(TensorNetwork(ψ))

We can find the energy of the system $|\psi\rangle=| + + \dots + \rangle$ by measuring the expectation value of the Hamiltonian $\langle \psi | H | \psi \rangle$.   

In [ ]:
using LinearAlgebra

λ = 0.3

ket₊ = 1/√2 * [1, 1]
ψ = Product(fill(ket₊, n))
ψ = convert(Chain, ψ)

function energy_Ising(ψ, λ)
    id = Matrix(1.0I, 2, 2)
    H = -1 * kron(Matrix(X()), Matrix(X())) + λ * (kron(Matrix(Z()), id) + kron(id, Matrix(Z())))
    H = reshape(H,2,2,2,2)

    e_i = []
    for i in 1:n-1
        H_gate = Dense(Qrochet.Operator(), H; sites=Site[Site(i), Site(i+1), Site(i, dual=true), Site(i+1, dual=true)])
        push!(e_i, only(expect(ψ, [H_gate])))
    end
    return e_i
end

energy_Ising(ψ, λ)

Now we can start the time evolution:

In [ ]:
ket₊ = 1/√2 * [1, 1]
ψ = Product(fill(ket₊, n))
ψ = convert(Chain, ψ)

# Time evolution:
expected_vals = Float64[]
@time for it in 1:timesteps
    # First apply the gates on even qubits
    for (i,j) in Iterators.filter(==(2) ∘ length, Iterators.partition(1:n,2))
        evolve!(ψ, trotter_XX(i,j; δₜ))
    end

    # Then apply the gates on odd qubits
    for (i,j) in Iterators.filter(==(2) ∘ length, Iterators.partition(2:n,2))
        evolve!(ψ, trotter_XX(i,j; δₜ))
    end

    # Finally apply the one-qubit gates
    for i in 1:n
        evolve!(ψ, trotter_Z(i; λ=0.3, δₜ))
    end

    # Compute the expectation value of the observable
    push!(expected_vals, abs(only(expect(ψ, observables))))
end

In [ ]:
# Plot the expectation value:
fig = Figure(fontsize=18)
ax = Axis(fig[1, 1], xlabel="time", ylabel="Observable")
lines!(ax, (1:timesteps) .* δₜ, expected_vals, color=:red, linestyle=:solid, marker=:circle, label="Exact result", fontsize=12)

legend = Legend(fig[1, 2], ax, valign=:top)
fig

But what if we had a large system? We can limit the bond dimension $\chi$! To do this, we first `canonize` the MPS:

In [ ]:
ket₊ = 1/√2 * [1, 1]
ψ_original = convert(Chain, Product(fill(ket₊, n)))

canonize!(ψ)
plot(TensorNetwork(ψ), layout=Stress())

In [ ]:
expectedvals = Dict()

ket₊ = 1/√2 * [1, 1]
ψ_original = convert(Chain, Product(fill(ket₊, n)))

observables = Dense.([Z(5)])

maxdims = [1, 2, 4, 8, 16]

for maxdim in maxdims
    # Reset ψ to its original state for each maxdim iteration
    ψ = deepcopy(ψ_original)
    canonize!(ψ)
    expectedvals_maxdim = Float64[]

    @time for it in 1:timesteps
        for (i,j) in Iterators.filter(==(2) ∘ length, Iterators.partition(1:n,2))
            evolve!(ψ, trotter_XX(i,j; δₜ); maxdim=maxdim, iscanonical=true)
        end
        for (i,j) in Iterators.filter(==(2) ∘ length, Iterators.partition(2:n,2))
            evolve!(ψ, trotter_XX(i,j; δₜ); maxdim=maxdim, iscanonical=true)
        end
        for i in 1:n
            evolve!(ψ, trotter_Z(i; λ=0.3, δₜ); maxdim=maxdim, iscanonical=true)
        end
        push!(expectedvals_maxdim, abs(only(expect(ψ, observables))))
    end

    expectedvals[maxdim] = expectedvals_maxdim
end

In [ ]:
using CairoMakie

# Assuming `expectedvals` and `maxdims` are defined as before

fig = Figure()
ax = Axis(fig[1, 1], xlabel="Timestep", ylabel="Observable", title="Observables for Different maxdim")

# Reverse the order of these arrays to match the reversed maxdims
color_palette = reverse([:red, :green, :blue, :purple, :orange])
line_styles = reverse([(:solid, 2), (:dash, 2), (:dot, 2), (:dashdot, 2), (:dashdotdot, 2)])
markers = reverse([:circle, :rect, :diamond, :cross, :star5])

# Reverse the order of iteration for maxdims
for (index, maxdim) in enumerate(maxdims)
    # Convert expectedvals[maxdim] to a Float32 array explicitly
    y_values = Float32.(expectedvals[maxdim])
    x_values = Float32.(1:length(y_values)) # Ensure x values are also Float32 for consistency

    lines!(ax, x_values, y_values,
          color=color_palette[index],
          linestyle=line_styles[index][1],
          linewidth=line_styles[index][2],
          marker=markers[index],
          markersize=10,
          label="maxdim = $maxdim")
end

legend = Legend(fig[1, 2], ax, valign=:top)
fig[1, 2] = legend

fig # Display the figure

### Exercise !

Use the `TEBD` algorithm to simulate the time evolution of a Matrix Product State (MPS) under the XXZ Hamiltonian: $H = -\frac{1}{2}\sum^{N}_{i}(\sigma_{i}^{x}\sigma_{i+1}^{x}+\sigma_{i}^{y}\sigma_{i+1}^{y}+\Delta\sigma_{i}^{z}\sigma_{i+1}^{z})$


Measure the magnetization of the system during the quench, which is defined as $M = \frac{1}{N} \sum_{i=1}^{N}\langle\sigma_{i}^{z}\rangle$.
Start with a system where all spins are in the $|\uparrow\rangle$ state. Use the `TEBD` algorithm with a bond dimension $\chi$ to simulate the time evolution of the system.

Tip: Start with a small system (e.g. $N=12$) and a small number of time steps to test your implementation.

- Plot the average magnetization of the system during the quench using `Makie`.
- Do a 2D plot of the magnetization per site $m_{i}$ of the system as a function of time and site. See https://docs.makie.org/stable/reference/plots/heatmap/#example_4a31556a for an example of how to plot a heatmap with `Makie`.
- How does the magnetization of the system change as a function of the bond dimension $\chi$?
- Compute the energy and verify that it is conserved during the time evolution. How does that change if we use a larger time step $\delta t$?